In [2]:
import sys, os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import pickle
from pathlib import Path



In [6]:
ROOT = Path.cwd().parents[0]
RESULT_FILE = ROOT / 'datamodel' / 'xgboost_panel_with_feat_label_enc_pred.csv'
ENCODER_FILE  = ROOT / 'datamodel' / 'xgboost_panel_with_feat_label_enc.pkl'
MODEL_FILE  = ROOT / 'datamodel' / 'xgboost_panel_with_feat.pkl'






In [ ]:
# Reload categorical encoders from file
with open(ENCODER_FILE, 'rb') as f:
    categorical_encoders = pickle.load(f)

# Reload model from file
with open(MODEL_FILE, 'rb') as f:
    categorical_encoders = pickle.load(f)



In [9]:
df=pd.read_csv(RESULT_FILE)